In [1]:
import tweepy
import numpy as np
import pandas as pd
import time
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

/home/brandon/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
consumer_token = "FSdktl3u8Q4xhmEHeC7tt4Q6P"
consumer_secret = "pErt27djL4HCcBt0z9PU6w7zVFygfVsEP1Aqx2gQ70uYibqP7p"
access_key = "1902120404-I1QEwvP7uLEBRJefMMPdPSf4uChDnsNvzO8hFcj"
access_secret = "RZAp4AlaXoszBXqp8gkMJM1ed6TJzTBFCDS99248hV9i4"

In [3]:
auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_key, access_secret)

In [4]:
api = tweepy.API(auth)

In [5]:
def makeCleanRep(origSource, removeLink=True):
    """
    Returns a list of the sid, text, and date of a given source.
    """
    
    sid = origSource._json['id']
    text = origSource._json['full_text']
    date = origSource._json['created_at']
    
    if removeLink==True:
        nonLinks = []
        for word in text.split(" "):
            if  "http" not in word:
                nonLinks.append(word)
        text = " ".join(nonLinks)
            
    
    return [sid, text, date]

In [6]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except (tweepy.RateLimitError, tweepy.TweepError):
            print("Rate limit exceeded, I'm passing out for a bit peace")
            time.sleep(15 * 60)

In [11]:
allData = []
ids = []
texts = []
dates = []

c = 0
for handle in ["technology", "business", "WSJbusiness", "BBCWorldBiz", "ForbesTech", "CNBCtech"]:
# for handle in ["technology"]:

    print("Starting", handle,  "...")
    hc = 0
#     cur = limit_handled(tweepy.Cursor(api.user_timeline, screen_name = "technology", tweet_mode='extended').items(5000))
    cur = tweepy.Cursor(api.user_timeline, screen_name = handle, tweet_mode='extended').items(3000)
    for source in limit_handled(cur):
#     for source in cur:
        clean = makeCleanRep(source)
    #     lowerTweet = clean[1].lower()
    #     if bname in lowerTweet:
    #         relevantTweetData.append(clean)
    #         print(clean)
        if clean[0] not in ids:
#             allData.append(clean)
            ids.append(clean[0])
            texts.append(clean[1])
            dates.append(clean[2])
            c += 1
            hc += 1
#             print(c, clean[0])


    #     if len(relevantTweetData) >= 10:
    #         break
    print("Gathered", hc, "tweets for", handle)

Starting technology ...


/home/brandon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'limit_handled' raised StopIteration
  


Gathered 3000 tweets for technology
Starting business ...
Gathered 3000 tweets for business
Starting WSJbusiness ...
Gathered 3000 tweets for WSJbusiness
Starting BBCWorldBiz ...
Gathered 3000 tweets for BBCWorldBiz
Starting ForbesTech ...
Gathered 3000 tweets for ForbesTech
Starting CNBCtech ...
Gathered 3000 tweets for CNBCtech


In [12]:
df = pd.DataFrame({"IDs":ids, "Texts":texts, "Dates":dates}, columns=["IDs", "Texts", "Dates"])
df

,IDs,Texts,Dates
0,1010341734780416001,We’re now on WhatsApp. Here’s how to get started,Sat Jun 23 02:00:15 +0000 2018
1,1010224968825745408,"The Uber backup driver was watching ""The Voice...",Fri Jun 22 18:16:16 +0000 2018
2,1010134917492170753,RT @JonErlichman: On this day in 2012: Tesla b...,Fri Jun 22 12:18:26 +0000 2018
3,1010094860743962624,RT @crypto: Bitcoin has biggest drop in more t...,Fri Jun 22 09:39:16 +0000 2018
4,1009979371686383616,Be one of the first to sign up,Fri Jun 22 02:00:21 +0000 2018
5,1009979366661607424,Intel CEO's ouster exposes a very thin bench,Fri Jun 22 02:00:20 +0000 2018
6,1009963104103550977,"Thanks to Xiaomi, China's big tech stock plan ...",Fri Jun 22 00:55:42 +0000 2018
7,1009958860868734976,Fujifilm has failed to persuade a Manhattan st...,Fri Jun 22 00:38:51 +0000 2018
8,1009928691760197633,Facebook's Sheryl Sandberg is addressing a gat...,Thu Jun 21 22:38:58 +0000 2018
9,1009919602330165248,YouTube is offering a talent option to sell su...,Thu Jun 21 22:02:51 +0000 2018


In [13]:
postDf = df.copy()
sia = SIA()

postDf['Sentiment'] = pd.Series([sia.polarity_scores(text)['compound'] for text in postDf['Texts']])
postDf

,IDs,Texts,Dates,Sentiment
0,1010341734780416001,We’re now on WhatsApp. Here’s how to get started,Sat Jun 23 02:00:15 +0000 2018,0.0000
1,1010224968825745408,"The Uber backup driver was watching ""The Voice...",Fri Jun 22 18:16:16 +0000 2018,-0.7579
2,1010134917492170753,RT @JonErlichman: On this day in 2012: Tesla b...,Fri Jun 22 12:18:26 +0000 2018,0.0000
3,1010094860743962624,RT @crypto: Bitcoin has biggest drop in more t...,Fri Jun 22 09:39:16 +0000 2018,-0.2732
4,1009979371686383616,Be one of the first to sign up,Fri Jun 22 02:00:21 +0000 2018,0.0000
5,1009979366661607424,Intel CEO's ouster exposes a very thin bench,Fri Jun 22 02:00:20 +0000 2018,-0.1280
6,1009963104103550977,"Thanks to Xiaomi, China's big tech stock plan ...",Fri Jun 22 00:55:42 +0000 2018,0.4404
7,1009958860868734976,Fujifilm has failed to persuade a Manhattan st...,Fri Jun 22 00:38:51 +0000 2018,-0.5106
8,1009928691760197633,Facebook's Sheryl Sandberg is addressing a gat...,Thu Jun 21 22:38:58 +0000 2018,0.0000
9,1009919602330165248,YouTube is offering a talent option to sell su...,Thu Jun 21 22:02:51 +0000 2018,0.4215


In [16]:
postDf.to_csv("historical.csv",  index=False)

In [18]:
loadedDf = pd.read_csv("historical.csv")
loadedDf['Dates']

0        Sat Jun 23 02:00:15 +0000 2018
1        Fri Jun 22 18:16:16 +0000 2018
2        Fri Jun 22 12:18:26 +0000 2018
3        Fri Jun 22 09:39:16 +0000 2018
4        Fri Jun 22 02:00:21 +0000 2018
5        Fri Jun 22 02:00:20 +0000 2018
6        Fri Jun 22 00:55:42 +0000 2018
7        Fri Jun 22 00:38:51 +0000 2018
8        Thu Jun 21 22:38:58 +0000 2018
9        Thu Jun 21 22:02:51 +0000 2018
10       Thu Jun 21 21:19:13 +0000 2018
11       Thu Jun 21 20:55:08 +0000 2018
12       Thu Jun 21 20:25:33 +0000 2018
13       Thu Jun 21 19:57:06 +0000 2018
14       Thu Jun 21 19:48:06 +0000 2018
15       Thu Jun 21 19:21:38 +0000 2018
16       Thu Jun 21 18:54:45 +0000 2018
17       Thu Jun 21 18:33:09 +0000 2018
18       Thu Jun 21 18:06:52 +0000 2018
19       Thu Jun 21 17:48:17 +0000 2018
20       Thu Jun 21 17:38:44 +0000 2018
21       Thu Jun 21 17:14:09 +0000 2018
22       Thu Jun 21 16:48:48 +0000 2018
23       Thu Jun 21 16:25:55 +0000 2018
24       Thu Jun 21 16:00:30 +0000 2018


In [ ]:
cleanTweetData[0]

In [ ]:
relevantTweetData[0]